In [1]:
import pandas as pd
import pyrankvote
from pathlib import Path
from random import seed
seed(252)

In [2]:
df = pd.read_csv(Path('./data/20210213.csv').resolve())
df

,Timestamp,Rank your movie choices. You don't have to do a full ranking. [First Choice],Rank your movie choices. You don't have to do a full ranking. [Second Choice],Rank your movie choices. You don't have to do a full ranking. [Third Choice],Rank your movie choices. You don't have to do a full ranking. [Fourth Choice],Rank your movie choices. You don't have to do a full ranking. [Fifth Choice],Rank your movie choices. You don't have to do a full ranking. [Sixth Choice],Rank your movie choices. You don't have to do a full ranking. [Seventh Choice],What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice],What theme should we explore next week? [Fourth Choice],What theme should we explore next week? [Fifth Choice],What theme should we explore next week? [Sixth Choice]
0,2/9/2021 19:09:39,Solaris (1972),2046 (2004),Trollhunter (2010),Aniara (2019),Attack the Block (2011),The City of Lost Children (1995),Violence Voyager (2018),IT'S TRUE I SAW IT IN A DOCUMENTARY,SEMI HISTORICALLY ACCURATE PERIOD PIECES,[Default to Nicolas Cage],BETTER THAN THE BOOK IT IS BASED ON,"BRO, THIS MOVIE IS FUCKED UP!",CINEMA ANTIFA
1,2/9/2021 19:15:49,Trollhunter (2010),Aniara (2019),Violence Voyager (2018),Solaris (1972),Attack the Block (2011),2046 (2004),The City of Lost Children (1995),[Default to Nicolas Cage],SEMI HISTORICALLY ACCURATE PERIOD PIECES,"BRO, THIS MOVIE IS FUCKED UP!",CINEMA ANTIFA,NaN,IT'S TRUE I SAW IT IN A DOCUMENTARY
2,2/9/2021 19:30:22,Solaris (1972),Trollhunter (2010),Aniara (2019),2046 (2004),Violence Voyager (2018),Attack the Block (2011),The City of Lost Children (1995),CINEMA ANTIFA,"BRO, THIS MOVIE IS FUCKED UP!",IT'S TRUE I SAW IT IN A DOCUMENTARY,SEMI HISTORICALLY ACCURATE PERIOD PIECES,NaN,BETTER THAN THE BOOK IT IS BASED ON
3,2/9/2021 20:29:18,Solaris (1972),2046 (2004),Aniara (2019),Trollhunter (2010),The City of Lost Children (1995),Violence Voyager (2018),NaN,CINEMA ANTIFA,BETTER THAN THE BOOK IT IS BASED ON,"BRO, THIS MOVIE IS FUCKED UP!",SEMI HISTORICALLY ACCURATE PERIOD PIECES,IT'S TRUE I SAW IT IN A DOCUMENTARY,[Default to Nicolas Cage]
4,2/9/2021 21:41:32,Attack the Block (2011),Aniara (2019),2046 (2004),Trollhunter (2010),The City of Lost Children (1995),Solaris (1972),Violence Voyager (2018),SEMI HISTORICALLY ACCURATE PERIOD PIECES,"BRO, THIS MOVIE IS FUCKED UP!",IT'S TRUE I SAW IT IN A DOCUMENTARY,BETTER THAN THE BOOK IT IS BASED ON,CINEMA ANTIFA,[Default to Nicolas Cage]
5,2/10/2021 15:29:02,Violence Voyager (2018),Aniara (2019),Attack the Block (2011),Solaris (1972),The City of Lost Children (1995),2046 (2004),Trollhunter (2010),SEMI HISTORICALLY ACCURATE PERIOD PIECES,BETTER THAN THE BOOK IT IS BASED ON,"BRO, THIS MOVIE IS FUCKED UP!",CINEMA ANTIFA,IT'S TRUE I SAW IT IN A DOCUMENTARY,[Default to Nicolas Cage]
6,2/11/2021 5:27:18,Attack the Block (2011),Aniara (2019),The City of Lost Children (1995),2046 (2004),NaN,NaN,NaN,SEMI HISTORICALLY ACCURATE PERIOD PIECES,IT'S TRUE I SAW IT IN A DOCUMENTARY,BETTER THAN THE BOOK IT IS BASED ON,CINEMA ANTIFA,"BRO, THIS MOVIE IS FUCKED UP!",NaN
7,2/11/2021 5:28:20,Attack the Block (2011),Trollhunter (2010),Aniara (2019),2046 (2004),NaN,NaN,NaN,SEMI HISTORICALLY ACCURATE PERIOD PIECES,IT'S TRUE I SAW IT IN A DOCUMENTARY,CINEMA ANTIFA,[Default to Nicolas Cage],NaN,NaN
8,2/11/2021 11:04:02,Violence Voyager (2018),Solaris (1972),The City of Lost Children (1995),Attack the Block (2011),Aniara (2019),2046 (2004),Trollhunter (2010),SEMI HISTORICALLY ACCURATE PERIOD PIECES,"BRO, THIS MOVIE IS FUCKED UP!",BETTER THAN THE BOOK IT IS BASED ON,IT'S TRUE I SAW IT IN A DOCUMENTARY,CINEMA ANTIFA,[Default to Nicolas Cage]


In [3]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [4]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array(['Solaris (1972)', '2046 (2004)', 'Trollhunter (2010)',
       'Aniara (2019)', 'Attack the Block (2011)',
       'The City of Lost Children (1995)', 'Violence Voyager (2018)'],
      dtype=object)

In [5]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [6]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [7]:
submission_dict

{'Solaris (1972)': <Candidate('Solaris (1972)')>,
 '2046 (2004)': <Candidate('2046 (2004)')>,
 'Trollhunter (2010)': <Candidate('Trollhunter (2010)')>,
 'Aniara (2019)': <Candidate('Aniara (2019)')>,
 'Attack the Block (2011)': <Candidate('Attack the Block (2011)')>,
 'The City of Lost Children (1995)': <Candidate('The City of Lost Children (1995)')>,
 'Violence Voyager (2018)': <Candidate('Violence Voyager (2018)')>}

In [8]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [9]:
ballots

[<Ballot(Solaris (1972), 2046 (2004), Trollhunter (2010), Aniara (2019), Attack the Block (2011), The City of Lost Children (1995), Violence Voyager (2018))>,
 <Ballot(Trollhunter (2010), Aniara (2019), Violence Voyager (2018), Solaris (1972), Attack the Block (2011), 2046 (2004), The City of Lost Children (1995))>,
 <Ballot(Solaris (1972), Trollhunter (2010), Aniara (2019), 2046 (2004), Violence Voyager (2018), Attack the Block (2011), The City of Lost Children (1995))>,
 <Ballot(Solaris (1972), 2046 (2004), Aniara (2019), Trollhunter (2010), The City of Lost Children (1995), Violence Voyager (2018))>,
 <Ballot(Attack the Block (2011), Aniara (2019), 2046 (2004), Trollhunter (2010), The City of Lost Children (1995), Solaris (1972), Violence Voyager (2018))>,
 <Ballot(Violence Voyager (2018), Aniara (2019), Attack the Block (2011), Solaris (1972), The City of Lost Children (1995), 2046 (2004), Trollhunter (2010))>,
 <Ballot(Attack the Block (2011), Aniara (2019), The City of Lost Child

In [10]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [11]:
print(election_result)

ROUND 1
Candidate                           Votes  Status
--------------------------------  -------  --------
Solaris (1972)                          3  Hopeful
Attack the Block (2011)                 3  Hopeful
Violence Voyager (2018)                 2  Rejected
Trollhunter (2010)                      1  Rejected
Aniara (2019)                           0  Rejected
2046 (2004)                             0  Rejected
The City of Lost Children (1995)        0  Rejected

FINAL RESULT
Candidate                           Votes  Status
--------------------------------  -------  --------
Solaris (1972)                          5  Elected
Attack the Block (2011)                 4  Rejected
Violence Voyager (2018)                 0  Rejected
Trollhunter (2010)                      0  Rejected
Aniara (2019)                           0  Rejected
2046 (2004)                             0  Rejected
The City of Lost Children (1995)        0  Rejected



In [12]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

FINAL RESULT
Candidate                                   Votes  Status
----------------------------------------  -------  --------
SEMI HISTORICALLY ACCURATE PERIOD PIECES        5  Elected
CINEMA ANTIFA                                   2  Rejected
IT'S TRUE I SAW IT IN A DOCUMENTARY             1  Rejected
[Default to Nicolas Cage]                       1  Rejected
BRO, THIS MOVIE IS FUCKED UP!                   0  Rejected
BETTER THAN THE BOOK IT IS BASED ON             0  Rejected

